<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Run Notebook on AzureML
## Introduction to Azure Machine Learning  
The **[Azure Machine Learning service (AzureML)](https://docs.microsoft.com/azure/machine-learning/service/overview-what-is-azure-ml)** provides a cloud-based environment you can use to prep data, train, test, deploy, manage, and track machine learning models. By using Azure Machine Learning service, you can start training on your local machine and then scale out to the cloud. With many available compute targets, like [Azure Machine Learning Compute](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute) and [Azure Databricks](https://docs.microsoft.com/en-us/azure/azure-databricks/what-is-azure-databricks), and with [advanced hyperparameter tuning services](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters), you can build better models faster by using the power of the cloud.

Data scientists and AI developers use the main [Azure Machine Learning Python SDK](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/intro?view=azure-ml-py) to build and run machine learning workflows with the Azure Machine Learning service. You can interact with the service in any Python environment, including Jupyter Notebooks or your favorite Python IDE. The Azure Machine Learning SDK allows you the choice of using local or cloud compute resources, while managing and maintaining the complete data science workflow from the cloud.
![AzureML Workflow](https://docs.microsoft.com/en-us/azure/machine-learning/service/media/overview-what-is-azure-ml/aml.png)

This noteboook provides a scaffold of how to directly submit an existing notebook to AzureML compute targets. 

### Advantages of using AzureML:
- Manage cloud resources for monitoring, logging, and organizing your machine learning experiments.
- Train models either locally or by using cloud resources, including GPU-accelerated model training.
- Easy to scale out when dataset grows - by just creating and pointing to new compute target

### Prerequisities
   - **Azure Subscription**
       -  You can [create a free subscription](https://azure.microsoft.com/en-us/free/) or access existing subscription information from the Azure portal. Later in this notebook you will need information such as your subscription ID in order to create and access AzureML workspaces.
          - New subscription comes with [free credit](https://azure.microsoft.com/en-us/free/free-account-faq/) which will easily cover the cost of running this example notebook

# Setup environment

### Install azure.contrib.notebook package
Since azureml.contrib.notebook contains experimental components, it's not included in generated .yaml file by default.

In [33]:
#!pip install "azureml.contrib.notebook>=1.0.21.1"

### Generate reco_base.yaml file

In [34]:
!python ../../scripts/generate_conda_file.py

Generated conda file: reco_base.yaml

To create the conda environment:
$ conda env create -f reco_base.yaml

To update the conda environment:
$ conda env update -f reco_base.yaml

To register the conda environment in Jupyter:
$ conda activate reco_base
$ python -m ipykernel install --user --name reco_base --display-name "Python (reco_base)"



In [41]:
import os
import azureml.core
from azureml.core import Workspace
from os import path, makedirs
import shutil
from azureml.core import Experiment
from azureml.widgets import RunDetails
from azureml.core.runconfig import RunConfiguration
from azureml.contrib.notebook.notebook_run_config import NotebookRunConfig, PapermillExecutionHandler
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

print("This notebook was created using version 1.0.21 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.0.21 of the Azure ML SDK
You are currently using version 1.0.21 of the Azure ML SDK


### Connect to an AzureML workspace

An [AzureML Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows. In particular, an Azure ML Workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, deployment, inferencing, and the monitoring of deployed models.

**If you are trying to create or connect to a workspace for the first time in this environment, uncomment and run the below cell. ** It will load code necessary to create workspace. After code is loaded, run the cell again to create the workspace. It will create `aml_config\config.json` file which contains workspace configuration for future notebook runs. See [this tutorial](https://docs.microsoft.com/en-us/azure/machine-learning/service/setup-create-workspace#portal) to locate information such as subscription id.

In [36]:
# %load ../../scripts/create_workspace.py

Wrote the config file config.json to: /data/home/testuser/notebooks/Recommenders/notebooks/00_quick_start/aml_config/config.json
AzureML workspace name:  setup-ws


In [37]:
ws = Workspace.from_config()

Found the config file in: /data/home/testuser/notebooks/Recommenders/notebooks/00_quick_start/aml_config/config.json


This example uses run-based AzureML Compute, which requires minimum setup. Alternatively, you can use persistent compute target (cpu or gpu cluster) or remote virtual machines. For more details, see [How to set up training targets](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets).

Cost-wise, according to Azure [Pricing calculator](https://azure.microsoft.com/en-us/pricing/calculator/), with example VM size `STANDARD_D2_V2`, it costs a few dollars to run this notebook, which is well covered by Azure new subscription credit. For billing and pricing questions, please contact [Azure support](https://azure.microsoft.com/en-us/support/options/).

**Note**:
- 10m and 20m dataset requires more capacity than `STANDARD_D2_V2`, such as `STANDARD_NC6` or `STANDARD_NC12`. See list of all available VM sizes [here](https://docs.microsoft.com/en-us/azure/templates/Microsoft.Compute/2018-10-01/virtualMachines?toc=%2Fen-us%2Fazure%2Fazure-resource-manager%2Ftoc.json&bc=%2Fen-us%2Fazure%2Fbread%2Ftoc.json#hardwareprofile-object).
- As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [these instructions](https://docs.microsoft.com/en-us/azure/azure-supportability/resource-manager-core-quotas-request) on the default limits and how to request more quota.



In [ ]:
NOTEBOOK_NAME = 'sar_movielens.ipynb'
EXPERIMENT_NAME = 'sdk-papermill'
exp_directory = 'test_dir' + EXPERIMENT_NAME
# create experiment directory
if not path.exists(exp_directory):
    makedirs(exp_directory)
notebook_path = path.join(exp_directory, NOTEBOOK_NAME)
shutil.copy(NOTEBOOK_NAME, notebook_path)
        
exp = Experiment(workspace=ws, name=EXPERIMENT_NAME)

run_config_aml = RunConfiguration()
run_config_aml.target = "amlcompute" # possible to use different compute targets
run_config_aml.amlcompute.vm_size = 'STANDARD_D2_V2'
run_config_aml.environment.docker.enabled = True
run_config_aml.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config_aml.environment.python.user_managed_dependencies = False
run_config_aml.auto_prepare_environment = True
run_config_aml.environment.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='reco_base.yaml')

cfg = NotebookRunConfig(source_directory='../../',
                            notebook='notebooks/00_quick_start/' + NOTEBOOK_NAME,
                            output_notebook='outputs/out.ipynb',
                            parameters={"MOVIELENS_DATA_SIZE": "100k", "TOP_K": 10},
                            run_config=run_config_aml)


`exp.submit()` will submit source_directory folder and designate notebook to run

In [ ]:
run = exp.submit(cfg)
run

Above cell should output similar table as below
![Experiment submit output](https://recodatasets.blob.core.windows.net/images/aml_papermill_cell_output.jpg)
After clicking "Link to Azure Portal", experiment run details tab looks like this
![Azure Portal Experiment](https://recodatasets.blob.core.windows.net/images/aml_papermill_portal.jpg)
Experiment output is out.ipynb. You can download and view this file locally.
![Output file](https://recodatasets.blob.core.windows.net/images/aml_papermill_outputs_tab.jpg)
#### Jupyter widget

Jupyter widget can watch the progress of the run.  Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [39]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…